In [1]:
%cd ~/Github/sspai_spider/analysis/
%matplotlib inline
%config InlineBackend.figure_format = 'retina' #如果使用的是 windows 可以替换成`svg`矢量图格式

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.sans-serif'] = 'simhei'
plt.rcParams['axes.unicode_minus'] = False

/Users/linxiaoyue/Github/sspai_spider/analysis


In [16]:
#读入 data 文件夹中的数据
page = pd.read_excel('../data/page_data.xlsx')
users = pd.read_excel('../data/user_data.xlsx')

In [17]:
print(page.info())
page.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12243 entries, 0 to 12242
Data columns (total 26 columns):
advertisement_url      10 non-null object
author.avatar          12033 non-null object
author.id              12243 non-null int64
author.nickname        12243 non-null object
author.slug            12243 non-null object
banner                 11913 non-null object
comment_count          12243 non-null int64
corner.color           119 non-null object
corner.icon            357 non-null object
corner.id              12243 non-null int64
corner.memo            357 non-null object
corner.name            357 non-null object
corner.url             0 non-null float64
free                   12243 non-null bool
id                     12243 non-null int64
important              12243 non-null int64
like_count             12243 non-null int64
morning_paper_title    12243 non-null object
post_type              12243 non-null int64
released_time          12243 non-null datetime64[ns]
series

,advertisement_url,author.avatar,author.id,author.nickname,author.slug,banner,comment_count,corner.color,corner.icon,corner.id,...,like_count,morning_paper_title,post_type,released_time,series,special_columns,status,summary,title,view_count
0,NaN,attachment/origin/2015/01/06/211491.jpg,0,李小明,edxmd87l,attachment/origin/87/00/00/87.jpg,0,NaN,NaN,0,...,1,[],1,2012-03-27 02:46:37,[],[],0,明天就是诺基亚正式在国内发布Windows Phone手机的日子，相信一定有很多WP党或者是...,诺基亚Lumia即将登陆，中国电信力推,2908
1,NaN,attachment/origin/2015/01/06/211491.jpg,0,李小明,edxmd87l,attachment/origin/90/00/00/90.jpg,0,NaN,NaN,0,...,2,[],1,2012-03-27 03:25:21,[],[],0,新 iPad 上拥有 500 万像素的背照式摄像头，摄影性能比上一代 iPad 要提高许多。...,Pix：iPad 上的拍照应用,4259
2,NaN,attachment/origin/2014/12/01/198940.jpg,0,laisoukin,82u1db40,attachment/origin/2015/02/12/225281.jpg,0,NaN,NaN,0,...,1,[],1,2012-03-27 05:42:00,[],[],0,在使用iPhone时如何有电话打进来，会有“拒绝”和“接听”两个选项供选择。 但是在待机状态...,iPhone4S基础教程：待机状态如何实现挂断电话,24486
3,NaN,attachment/origin/2014/12/01/198940.jpg,0,laisoukin,82u1db40,attachment/origin/2015/02/12/225280.jpg,0,NaN,NaN,0,...,1,[],1,2012-03-27 07:32:00,[],[],0,在如何添加/删除键盘这篇文章里，我们了解了如何在系统键盘中添加适合自己的键盘，来提高我们的输...,iPhone4S基础教程：如何快速切换键盘,13659
4,NaN,attachment/origin/2015/02/13/225448.jpg,0,地瓜,nc0ulnjo,attachment/origin/92/00/00/92.jpg,0,NaN,NaN,0,...,1,[],1,2012-03-27 08:16:37,[],[],0,"这次愤怒的小鸟以全新的姿态出现在了我们眼前,感觉更像一支战队.小鸟们遇到了一只外星鸟,外星鸟...",欢迎回来《愤怒的小鸟太空版》...,6113


In [18]:
print(users.info())
users.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679 entries, 0 to 678
Data columns (total 6 columns):
author_slug      679 non-null object
create_at        679 non-null int64
followers_num    679 non-null int64
like_count       679 non-null int64
view_count       679 non-null int64
word_count       679 non-null int64
dtypes: int64(5), object(1)
memory usage: 31.9+ KB
None


,author_slug,create_at,followers_num,like_count,view_count,word_count
0,edxmd87l,1330906220,0,11,43857,6463
1,82u1db40,1330883844,2,124,4561484,137900
2,nc0ulnjo,1332698004,1,57,258034,103329
3,w5si9qrh,1332716184,3,41,774311,135141
4,michael,1330906250,471,3413,1022730,160287


# 数据处理部分

## 首页数据预处理

首页数据由于字段比较多，但是并非都是我们需要的，因此我们仅仅只需要聚焦那么一些需要的字段，其他的则选择忽略。这里我主要选取了九个字段：

- id：文章 id
- title：标题
- post_type：类型
- free：是否付费
- author_slug：作者域名（用来关联用户成就数据）
- author_nickname：作者昵称
- released_time：发布时间
- comment_count：评论数
- like_count：点赞数/充电数
- view_count：观看数

In [19]:
cols = [
    'id', 'title', 'post_type', 'free',
    'author.slug', 'author.nickname', 'released_time',
    'comment_count', 'like_count', 'view_count'
]
page = page[cols]
page['released_time'] = pd.to_datetime(page['released_time'])
page.columns = page.columns.str.replace('.', '_')

In [20]:
page.head()

,id,title,post_type,free,author_slug,author_nickname,released_time,comment_count,like_count,view_count
0,1541,诺基亚Lumia即将登陆，中国电信力推,1,True,edxmd87l,李小明,2012-03-27 02:46:37,0,1,2908
1,1561,Pix：iPad 上的拍照应用,1,True,edxmd87l,李小明,2012-03-27 03:25:21,0,2,4259
2,1570,iPhone4S基础教程：待机状态如何实现挂断电话,1,True,82u1db40,laisoukin,2012-03-27 05:42:00,0,1,24486
3,1586,iPhone4S基础教程：如何快速切换键盘,1,True,82u1db40,laisoukin,2012-03-27 07:32:00,0,1,13659
4,1579,欢迎回来《愤怒的小鸟太空版》...,1,True,nc0ulnjo,地瓜,2012-03-27 08:16:37,0,1,6113


查看一下是否有缺失值，如果有则进行处理，如果没有则说明数据比较完整

In [21]:
page.isnull().any()

id                 False
title              False
post_type          False
free               False
author_slug        False
author_nickname    False
released_time      False
comment_count      False
like_count         False
view_count         False
dtype: bool